In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--jars /data/home/pavel.klemenkov/jars/kafka-clients-1.0.1.jar,/data/home/pavel.klemenkov/jars/spark-sql-kafka-0-10_2.11-2.3.2.jar --conf spark.sql.catalogImplementation=in-memory pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [5]:
KAFKA_SERVERS = (
    "master.cluster-lab.com:6667",
)

In [7]:
messages = spark.readStream\
                .format("kafka")\
                .option("kafka.bootstrap.servers", ",".join(KAFKA_SERVERS))\
                .option("subscribe", "streaming")\
                .load()

In [8]:
messages.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [9]:
import pyspark.sql.functions as f

In [10]:
words = messages.select(f.split(messages.value.cast("string"), " ").alias("words"))\
                .select(f.explode("words").alias("word"))

In [11]:
wordCounts = words.groupBy("word").count()

In [12]:
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("wordcount")\
    .trigger(processingTime="5 seconds")\
    .start()

In [20]:
spark.sql("select * from wordcount").show()

+--------+-----+
|    word|count|
+--------+-----+
|Goodbye,|    1|
|  world!|    2|
|   cruel|    1|
|  Hello,|    1|
+--------+-----+



In [21]:
query.isActive

True

In [22]:
query.stop()

In [23]:
spark.stop()